# import brightway2 results for the coal exit scenarios
0. calcualte the energy market electricity high voltage mix RECIPE total, HH, ED and RD
(indicator x scenarios x period)

0. calculate the mid to endpoint DALYS, and species.year

0. do the same for the other SE carriers 
    1. coarse match of REMIND and brightway technologies on REMIND SE level
    2. calculate the RECIPE total (3) for all these technologies in table form
       (technologies x indicator x scenarios x period)

1. use these tables to do a post processing (R) to aggregate and monetize the DALYS and monetize the ED
     1. read in results of this script
     2. use conversion factors for ecoinvent to REMIND
     3. use a REMIND activity mapping
     4. monetize DALYS and PDFs
    
    
further developments
1. adjust mid to end pop with SSP pop data country specific?
2. adjust the land occupation indicator with country specific species data
3. disaggregate the land occupating indicator 
4. update to RECIPE 2016 when available
5. dont use the global mean for SE technologies
6. automatic scenario name mapping

the se regional mapping is global

# import the brightway2 results for the coal exit scenarios

In [6]:
import os
import pandas as pd

In [7]:
path="C:\\Users\\rauner\\Documents\\PIK\\holi_coal_exit\\rmnd-lca\\"
os.chdir(path)

In [8]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [9]:
import brightway2 as bw
%run notebooks\\initialize_notebook.ipynb
from wurst.ecoinvent import filters
from constructive_geometries import Geomatcher

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\constructive_geometries\cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [10]:
from wurst import searching as ws

In [11]:
from helpers import eimod, lcahelp, ei2rmnd

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\constructive_geometries\cg.py:77: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():
C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\country_converter\country_converter.py:412: FutureWarning: read_table is deprecated, use read_csv instead.
  for str_col in must_be_string})


Geomatcher: Used 'AO' for 'AGO'
Geomatcher: Used 'BI' for 'BDI'
Geomatcher: Used 'BJ' for 'BEN'
Geomatcher: Used 'BF' for 'BFA'
Geomatcher: Used 'BW' for 'BWA'
Geomatcher: Used 'CF' for 'CAF'
Geomatcher: Used 'CI' for 'CIV'
Geomatcher: Used 'CM' for 'CMR'
Geomatcher: Used 'CD' for 'COD'
Geomatcher: Used 'CG' for 'COG'
Geomatcher: Used 'KM' for 'COM'
Geomatcher: Used 'CV' for 'CPV'
Geomatcher: Used 'DJ' for 'DJI'
Geomatcher: Used 'ER' for 'ERI'
Geomatcher: Used 'ET' for 'ETH'
Geomatcher: Used 'GA' for 'GAB'
Geomatcher: Used 'GH' for 'GHA'
Geomatcher: Used 'GN' for 'GIN'
Geomatcher: Used 'GM' for 'GMB'
Geomatcher: Used 'GW' for 'GNB'
Geomatcher: Used 'GQ' for 'GNQ'
Geomatcher: Used 'KE' for 'KEN'
Geomatcher: Used 'LR' for 'LBR'
Geomatcher: Used 'LS' for 'LSO'
Geomatcher: Used 'MG' for 'MDG'
Geomatcher: Used 'ML' for 'MLI'
Geomatcher: Used 'MZ' for 'MOZ'
Geomatcher: Used 'MR' for 'MRT'
Geomatcher: Used 'MU' for 'MUS'
Geomatcher: Used 'MW' for 'MWI'
Geomatcher: Used 'YT' for 'MYT'
Geomatch

In [12]:
bw.restore_project_directory("data\\brightway2-project-coal_exit-backup.08-January-2019-12-24PM.tar.gz")

Restoring project backup archive - this could take a few minutes...


'coal_exit'

In [13]:
bw.projects.set_current("coal_exit")
project = 'coal_exit'

## create a scenarios dictionary

In [14]:
database_dict = {}
database_dict['ecoinvent_added_technologies'] = {'year' : 2015 , 'scenario' : 'ecoinvent'}

for year in [ 2020, 2030,2040, 2050]:
    for scenario in ['Ref','RCP26','INDC','coal_exit']:
        if year == 2015 and scenario != 'ecoinvent_added_technologies':
            continue
        db_name = 'coal_exit_' + scenario + '_' + str(year)
        database_dict[db_name] = {'year' : year , 'scenario' : scenario}

In [15]:
database_dict

{'ecoinvent_added_technologies': {'year': 2015, 'scenario': 'ecoinvent'},
 'coal_exit_Ref_2020': {'year': 2020, 'scenario': 'Ref'},
 'coal_exit_RCP26_2020': {'year': 2020, 'scenario': 'RCP26'},
 'coal_exit_INDC_2020': {'year': 2020, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2020': {'year': 2020, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2030': {'year': 2030, 'scenario': 'Ref'},
 'coal_exit_RCP26_2030': {'year': 2030, 'scenario': 'RCP26'},
 'coal_exit_INDC_2030': {'year': 2030, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2030': {'year': 2030, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2040': {'year': 2040, 'scenario': 'Ref'},
 'coal_exit_RCP26_2040': {'year': 2040, 'scenario': 'RCP26'},
 'coal_exit_INDC_2040': {'year': 2040, 'scenario': 'INDC'},
 'coal_exit_coal_exit_2040': {'year': 2040, 'scenario': 'coal_exit'},
 'coal_exit_Ref_2050': {'year': 2050, 'scenario': 'Ref'},
 'coal_exit_RCP26_2050': {'year': 2050, 'scenario': 'RCP26'},
 'coal_exit_INDC_2050': {'year': 2050, 'scenario': 'INDC

## calcualte the LCA for all scenarios and years for all SE technologies

In [16]:
#select the modeled processes
#this is a crude mapping of of REMIND SE to ecoinvent progcesses
#we need to adjust the units later in the post processing to match EJ

In [17]:
# see data\ecoinventtoREMINDtechnmap.csv for SE technology mapping
# we use the electricity generation processes as a base and check what the input is to adjust them accordingly
# see the A matrix of the elec processes for the input A.technology.final.technology.list.variable.csv
# ergo if a kWh hard coal needs 5kg of coal we:

# elec to raw input = /5
# 1kWh to 1EJ =  /3.6*10^12
# elec hard coal to se hard coal /3.6*10^12 /5kg

# this factor is in the conv column of the ecoinventtoREMINDtechmap.csv

In [18]:
# se _filter filters for electricity high voltage and all the other SE ecoinvent processes mapped above

In [19]:
se_filter = [ws.either(*[                                                  
                                                                           ws.equals('name', 'heat and power co-generation, biogas, gas engine'),
                                                                           ws.equals('name', 'electricity production, natural gas, combined cycle power plant'),
                                                                           ws.equals('name', 'electricity production, hard coal'),
                                                                           ws.equals('name', 'electricity production, deep geothermal'),
                                                                           ws.equals('name', 'market for hydrogen, liquid'),
                                                                           ws.equals('name', 'electricity production, oil'),
                                                                           ws.equals('name', 'softwood forestry, mixed species, sustainable forest management')])]

In [20]:
electricity_markets = {}
se_technologies = {}

for db_name in database_dict:
    electricity_markets[db_name] = [x for x in ws.get_many(Database(db_name), *[*eimod.electricity_market_filter_high_voltage, ws.exclude(ws.contains('name', 'Swiss Federal Railways')),
                                                                                ws.exclude(ws.contains('name', 'label-certified'))])]
    se_technologies[db_name] = [x for x in ws.get_many(Database(db_name), *[*se_filter])]

## choose the RECIPE categories, we use 2008 with updated 2016 where direct matches are possible

calculate all the ReCiPe Midpoint (H) 2008 version and multiply with mid to endpoints to get DALYS (HH), Species.year (ET) and maybe resources.

In [21]:
method=[m for m in methods if "'ReCiPe Midpoint (H)'," in str(m)]
method

[('ReCiPe Midpoint (H)', 'agricultural land occupation', 'ALOP'),
 ('ReCiPe Midpoint (H)', 'climate change', 'GWP100'),
 ('ReCiPe Midpoint (H)', 'fossil depletion', 'FDP'),
 ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
 ('ReCiPe Midpoint (H)', 'freshwater eutrophication', 'FEP'),
 ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
 ('ReCiPe Midpoint (H)', 'ionising radiation', 'IRP_HE'),
 ('ReCiPe Midpoint (H)', 'marine ecotoxicity', 'METPinf'),
 ('ReCiPe Midpoint (H)', 'marine eutrophication', 'MEP'),
 ('ReCiPe Midpoint (H)', 'metal depletion', 'MDP'),
 ('ReCiPe Midpoint (H)', 'natural land transformation', 'NLTP'),
 ('ReCiPe Midpoint (H)', 'ozone depletion', 'ODPinf'),
 ('ReCiPe Midpoint (H)', 'particulate matter formation', 'PMFP'),
 ('ReCiPe Midpoint (H)', 'photochemical oxidant formation', 'POFP'),
 ('ReCiPe Midpoint (H)', 'terrestrial acidification', 'TAP100'),
 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
 ('ReCiPe Midpoint (H)', 'urban land 

select the matching categories, we only use the IPCC CC instead of the RECIPE one, however they have the same unit so we can use the same mid to endpoint factor

we use the RECIPE 1.11
    1. we developed a land occupation method ourselves to include it in the Ecosystem toxicity
    2. we did a crude matching of land occupation categories to urban and rural
    3. we took the average of each of the two categories
    4. for land transformation we use to and from forest and take the average terrestrial species per m2

In [22]:
categories = ['ALO', 'CC', 'FD', 'FET', 'FE',
              'HT', 'IR', 'MET', 'ME', 'MD',
              'NLT', 'OD', 'PMF', 'POF', 'TA',
              'TET', 'ULO', 'WD']
categories_total = ['ALO', 'CC', 'FD', 'FET', 'FE',
              'HT', 'IR', 'MET', 'ME', 'MD',
              'NLT', 'OD', 'PMF', 'POF', 'TA',
              'TET', 'ULO', 'WD','total'] 

perform LCA for all elec markets

In [23]:
electricity_markets_LCIA= {}
se_technologies_LCIA= {}

for db_name in database_dict:
    for category in categories:
        electricity_markets_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)] = lcahelp.LCA_to_df(electricity_markets[db_name], cats = [category])
        se_technologies_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)]     = lcahelp.LCA_to_df(se_technologies[db_name], cats = [category])

ALO


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


ALO


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


CC


...

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


WD


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


WD


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


load the mid to end characterization factors
calculate all results and aggregate them to DALYS and ET

In [24]:
mid_end = pd.read_csv("data\\recipe_mid_to_end.csv", sep = ';', index_col = 0) 
mid_end


,Midpoint impact category,Unit,HH_H,ED_H,RA_H
Code,,,,,
CC,Climate change,kg CO2 eq,9.280000e-07,2.800000e-09,0.0000
OD,Ozone depletion,kg CFC-11 eq,5.310000e-04,3.030000e-03,0.0000
TA,Terrestrial acidification,kg SO2 eq,0.000000e+00,2.120000e-07,0.0000
FE,Freshwater eutrophication,kg P eq,0.000000e+00,4.440000e-08,0.0000
HT,Human toxicity,"kg 1,4-DB eq",7.000000e-07,0.000000e+00,0.0000
POF,Photochemical oxidant formation,kg NMVOC,3.900000e-08,0.000000e+00,0.0000
PMF,Particulate matter formation,kg PM10 eq,2.600000e-04,0.000000e+00,0.0000
TET,Terrestrial ecotoxicity,"kg 1,4-DB eq",0.000000e+00,1.510000e-07,0.0000
FET,Freshwater ecotoxicity,"kg 1,4-DB eq",0.000000e+00,8.610000e-10,0.0000


In [25]:
end_categories = ['HH_H', 'ED_H', 'RA_H'] 
    

In [26]:
end_mid_electricity_markets = {}
end_mid_se_technologies = {}
for db_name in database_dict:        
    for end_category in end_categories:
        for category in categories:
                    
            end_mid_electricity_markets[(database_dict[db_name]['year'], database_dict[db_name]['scenario'],  end_category, category)] = electricity_markets_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)]
            end_mid_se_technologies[(database_dict[db_name]['year'], database_dict[db_name]['scenario'],  end_category, category)]     = se_technologies_LCIA[(database_dict[db_name]['year'], database_dict[db_name]['scenario'], category)]                                                                               


In [29]:
end_end_electricity_markets = pd.concat(end_mid_electricity_markets, names=['period','scenario','end_category', 'category', 'variable'])
end_end_se_technologies     = pd.concat(end_mid_se_technologies, names=['period','scenario','end_category', 'category', 'variable'])

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


sum up all end categories

In [30]:
end_end_electricity_markets = end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable']).sum()
end_end_se_technologies     = end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable']).sum()

In [31]:
end_end_electricity_markets['total'] = end_end_electricity_markets.sum(axis = 1)
end_end_se_technologies['total']     = end_end_se_technologies.sum(axis = 1)

In [32]:
end_end_electricity_markets.to_csv('interm\\lca_electricity_markets_per_tec_end_points' + project + '.csv', sep=';')
end_end_se_technologies.to_csv('interm\\lca_se_technologies_per_tec_end_points' + project + '.csv', sep=';')

In [33]:
end_end_electricity_markets  = pd.read_csv('interm\\lca_electricity_markets_per_tec_end_points' + project + '.csv', sep = ';') 
end_end_se_technologies      = pd.read_csv('interm\\lca_se_technologies_per_tec_end_points' + project + '.csv', sep = ';') 

add location

In [34]:
end_end_electricity_markets['ecoinvent'] = end_end_electricity_markets['variable'].str.split(",").str[2].str.split("'").str[1].str.split("-").str[0]
end_end_se_technologies['ecoinvent']     = end_end_se_technologies['variable'].str.split(",").str[-1].str.split("'").str[1].str.split("-").str[0]

match REMIND locations

In [35]:
end_end_electricity_markets['variable'] = end_end_electricity_markets['variable'].str.split(",").str[0].str.split("'").str[1]
end_end_se_technologies['variable'] = end_end_se_technologies['variable'].str.split("'").str[1]

In [36]:
region_mapping_elec     = pd.read_csv("data\\regionmappingREMIND_noFullISO_elec.csv", sep = ';', index_col = 0)
region_mapping_se       = pd.read_csv("data\\regionmappingREMIND_noFullISO_se.csv", sep = ';', index_col = 0)
region_mapping_se_world = pd.read_csv("data\\regionmappingREMIND_noFullISO_se_world.csv", sep = ';', index_col = False)

In [37]:
end_end_electricity_markets = pd.merge(end_end_electricity_markets, region_mapping_elec, on= "ecoinvent", how= "inner")
end_end_se_technologies = pd.merge(end_end_se_technologies, region_mapping_se, on= "ecoinvent", how= "inner")

average over multiple ecoinvent locations, we have India with several grids for example

In [38]:
#end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable','ecoinvent', 'CountryCode', 'RegionCode']).mean()
#end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable','ecoinvent', 'CountryCode', 'RegionCode']).mean()

In [39]:
end_end_electricity_markets = end_end_electricity_markets.groupby(['period','scenario', 'end_category', 'variable', 'RegionCode']).mean()
end_end_se_technologies = end_end_se_technologies.groupby(['period','scenario', 'end_category', 'variable', 'RegionCode']).mean()

## put the world values here for all REMIND regions for the non electricity se technologies

In [40]:
end_end_se_technologies.to_csv('interm\\lca_mid_points.csv', sep=';')
end_end_se_technologies = pd.read_csv("interm\\lca_mid_points.csv", sep = ';') 
end_end_electricity_markets.to_csv('interm\\lca_mid_points.csv', sep=';')
end_end_electricity_markets = pd.read_csv("interm\\lca_mid_points.csv", sep = ';')

In [41]:
end_end_se_technologies = pd.merge(end_end_se_technologies, region_mapping_se_world,  left_on=['RegionCode'], right_on=['World'], how= "right")

In [42]:
end_end_se_technologies['RegionCode'] = end_end_se_technologies['Remind']
#delte World and -- column
#end_end_se_technologies = end_end_se_technologies.drop('World', 1)
end_end_se_technologies = end_end_se_technologies.drop('World', 1)
end_end_se_technologies = end_end_se_technologies.drop('Remind', 1)

In [43]:
end_end = end_end_electricity_markets.append(end_end_se_technologies)

In [44]:
pd.DataFrame(end_end)

,period,scenario,end_category,variable,RegionCode,ALO,CC,FD,FE,FET,...,MET,NLT,OD,PMF,POF,TA,TET,ULO,WD,total
0,2015,ecoinvent,ED_H,market for electricity,AFR,0.001729,0.439718,0.150836,0.000115,0.002545,...,0.002611,0.000095,4.198364e-08,0.000892,0.001449,0.002575,0.000032,0.001414,0.000932,0.730810
1,2015,ecoinvent,ED_H,market for electricity,CHN,0.011656,0.878965,0.213994,0.000275,0.006721,...,0.006414,0.000043,9.639478e-09,0.001888,0.002891,0.004398,0.000027,0.008280,0.002890,1.428306
2,2015,ecoinvent,ED_H,market for electricity,EUR,0.054339,0.569361,0.170895,0.000550,0.009180,...,0.008971,0.000087,5.997834e-08,0.001279,0.001548,0.003521,0.000029,0.002380,0.002569,1.391724
3,2015,ecoinvent,ED_H,market for electricity,IND,0.016062,1.280238,0.391177,0.000632,0.011600,...,0.011009,0.000144,1.741131e-08,0.003447,0.003423,0.004719,0.000026,0.011272,0.004488,2.246514
4,2015,ecoinvent,ED_H,market for electricity,JPN,0.026135,0.757570,0.228966,0.000141,0.004030,...,0.003059,0.000270,2.431217e-08,0.000947,0.001913,0.003083,0.000042,0.003885,0.001848,1.179258
5,2015,ecoinvent,ED_H,market for electricity,LAM,0.031180,0.464673,0.158504,0.000060,0.001719,...,0.001839,0.000143,5.399119e-08,0.001132,0.001576,0.002649,0.000042,0.002528,0.000922,0.771723
6,2015,ecoinvent,ED_H,market for electricity,MEA,0.001667,0.652324,0.246513,0.000063,0.002090,...,0.002026,0.000159,5.694385e-08,0.000804,0.001613,0.002568,0.000039,0.001522,0.001714,1.029387
7,2015,ecoinvent,ED_H,market for electricity,OAS,0.005965,0.621784,0.198495,0.000414,0.007612,...,0.007177,0.000076,2.610363e-08,0.002857,0.001562,0.002577,0.000020,0.003618,0.002253,1.184256
8,2015,ecoinvent,ED_H,market for electricity,ROW,0.007997,0.492364,0.145093,0.000442,0.007515,...,0.007246,0.000105,3.359243e-08,0.001328,0.001498,0.003302,0.000025,0.001973,0.002387,1.075239
9,2015,ecoinvent,ED_H,market for electricity,RUS,0.006503,0.688144,0.259432,0.000216,0.004316,...,0.003815,0.000145,5.436389e-08,0.001113,0.001363,0.001946,0.000016,0.001889,0.002682,1.269723


join with REMIND mapping and energy content conversion

In [45]:
# join with REMIND mapping
## read REMIND mapping
## join
## divide with convertion to go to EJ

In [46]:
tech_mapping = pd.read_csv("data\\ecoinventtoREMINDtechmap.csv", sep = ';', index_col = False) 

In [47]:
end_end = pd.merge(end_end, tech_mapping, left_on=['variable'], right_on=['ecoinvent'], how= "left")

In [48]:
end_end['total'] = end_end['total'] / end_end['conv']
end_end[categories]  = end_end[categories].divide(end_end["conv"], axis="index")

cleanup

In [49]:
end_end_out = end_end[['period','scenario','REMIND','end_category', 'RegionCode']]
end_end_out[categories] = end_end[categories]
end_end_out['total'] = end_end['total']

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\frame.py:3381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [50]:
end_end_out = pd.melt(end_end_out, id_vars=['period', 'scenario','REMIND','RegionCode','end_category'], value_vars=categories_total)

write results

In [51]:
end_end_out.to_csv('results\\lca_per_tec_mid_points' + project  + '.csv', sep=';')

# read all REMIND mifs and calculate the actual model specific LCA including the monetization
1. read all mifs and append them
2. join with end_end_out and multiply
3. join with vsl_coeff
    1. join vsl_coeff with regions mapping and average over region

read REMIND results

In [52]:
mif_names = os.listdir("data\\mifs\\")

mifs from wide to long

In [53]:
mifs = pd.concat( [ pd.read_csv("data\\mifs\\" + mif, sep = ';') for mif in mif_names ] )

define colnames that are years

In [54]:
mifs = mifs.drop('Unnamed: 24', 1)

In [55]:
year_cols = [col for col in mifs.columns if '2' in col]

In [56]:
mifs = pd.melt(mifs, id_vars=['Model', 'Scenario','Region','Variable', 'Unit'], value_vars=year_cols)

rename columns

In [57]:
mifs.columns=['model','scenario','region','tech','unit','period','SE']
mifs = mifs.drop('model', 1)
end_end_out.columns=['period','scenario','tech','region','end_category','category','lca']

join LCA results with the mifs

In [58]:
mifs['period']=mifs['period'].astype(int)
end_end_out['period']=end_end_out['period'].astype(int)

map scenarios

In [59]:
end_end_out['scenario'][end_end_out['scenario'] == 'ecoinvent'] = 'SSP2-Ref'
end_end_out['scenario'][end_end_out['scenario'] == 'INDC'] = 'SSP2-INDC'
end_end_out['scenario'][end_end_out['scenario'] == 'RCP26'] = 'SSP2-26'
end_end_out['scenario'][end_end_out['scenario'] == 'Ref'] = 'SSP2-Ref'
end_end_out['scenario'][end_end_out['scenario'] == 'coal_exit'] = 'SSP2-INDC_coal_exit'

C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rauner\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from

In [60]:
end_end_out = pd.merge(end_end_out, mifs, left_on=['period','scenario','tech','region'], right_on=['period','scenario','tech','region'], how= "left")
end_end_out['lca_impact'] = end_end_out['lca'] * end_end_out['SE']

write results

In [61]:
end_end_out.to_csv('results\\lca_REMIND_mid_points' + project  + '.csv', sep=';')